In [1]:
import json
import glob
import sys, os, shutil
import numpy as np
from tqdm.auto import tqdm 

sys.path.append('/home/daniel/research/catkin_ws/src/')
from grub_nempc import run_grub_nempc, nempc_setup

In [2]:
#### SETUP ############
# Test Parameters
test_runs = 100

# Initial Conditions
p0 = np.array([[200],
                [200],
                [200],
                [200]])
qd0 = np.zeros((2,1))
q0 = np.random.uniform(low=-np.pi/2, high=np.pi/2, size=(test_runs, 2)) # random joint angle goal
p_goal = np.array([[0], [0], [0], [0]])
qd_goal = np.zeros((2,1))
ugoal = np.zeros([4, 1]) # right now in the cost function this is weighted at 0
q_goal = np.random.uniform(low=-np.pi/2, high=np.pi/2, size=(test_runs, 2)) # random joint angle goal

In [3]:
iae_metrics = []
trial_names = []
control_successes = []
INPUT_DIR = '/home/daniel/research/catkin_ws/src/hyperparam_optimization/bellows_grub/run_logs/'

run_logs = os.listdir(INPUT_DIR)

# Try the controller with the model============
control_results = np.zeros([test_runs,])
iae_results = np.zeros([test_runs,])


for run in run_logs:
    for trial in os.listdir(INPUT_DIR+run):
        if os.path.isdir(INPUT_DIR + run + '/' + trial):
            trial_dir = INPUT_DIR + run + '/' + trial
            trial_name = trial[:14]

            try:
                controller, ground_truth, _, u0, _, sim_length = nempc_setup(trial_dir=trial_dir)
                curr_control_success = []
                print(f'\n\nRunning Control for Trial: {trial_name}')

                for test in tqdm(range(test_runs)):
                    xgoal = np.vstack([p_goal,qd_goal,q_goal[test].reshape(2,1)])
                    x0 = np.vstack([p0,qd0,q0[test].reshape(2,1)])
                    x, iae_score, control_successful = run_grub_nempc(controller, ground_truth, x0, u0, xgoal, sim_length)
                    curr_control_success.append(control_successful)
                    control_results[test] = control_successful
                    iae_results[test] = iae_score
                
                iae_metrics.append(np.mean(iae_results))
                trial_names.append(trial_name+'/'+run)
                if np.mean(control_results) > 0.5:
                    print(f'Ran control successfully for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')
                else:
                    print(f'Control failed for trial: {trial_name}. Control Results: {np.mean(control_results)}, Average IAE={np.mean(iae_results)}')
                    raise ValueError

            except Exception as e:
                print(f'For trial {trial_name} got {e}')
                # shutil.rmtree(trial_dir)
                continue # no checkpoint, wasn't a good run

min_index = iae_metrics.index(min(iae_metrics))
print()
print(f'Best Trial: {trial_names[min_index]}, with IAE={iae_metrics[min_index]}')
print(f'Total Trials ran: {len(iae_metrics)}')



Running Control for Trial: train_e8d1e215


  0%|          | 0/100 [00:00<?, ?it/s]

Ran control successfully for trial: train_e8d1e215. Control Results: 0.55, Average IAE=286.4445821904852


Running Control for Trial: train_0db45b06


  0%|          | 0/100 [00:00<?, ?it/s]

Ran control successfully for trial: train_0db45b06. Control Results: 0.61, Average IAE=273.4296128426911


Running Control for Trial: train_849897e4


...

  0%|          | 0/100 [00:00<?, ?it/s]

Ran control successfully for trial: train_595eaec2. Control Results: 0.74, Average IAE=273.85194601962297


Running Control for Trial: train_a10c35ce


  0%|          | 0/100 [00:00<?, ?it/s]

Ran control successfully for trial: train_a10c35ce. Control Results: 0.56, Average IAE=282.0603933928994


Running Control for Trial: train_11940ae4


  0%|          | 0/100 [00:00<?, ?it/s]